In [1]:
import pandas as pd
import json
import os
from collections import Counter
import re
from dateutil.parser import parse
import xmltodict
import yaml
import sys
import glob
import ast
from datetime import datetime
from dataproc import preprocess
from tqdm import tqdm
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import gzip

In [ ]:
path='/HDD16TB/weisi/amazon_us_reviews/Health_Personal_Care_v1_00/0.1.0/reviews_Health_and_Personal_Care_5.json.gz'

with gzip.open(path,'rt') as f:
    df=pd.read_json(f,lines=True)

df.head

In [ ]:
print(df.iloc[0])
print(df.at[0,'reviewText'])

In [ ]:
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
df.reviewText = df.reviewText.progress_apply(lambda x: preprocess.clean_and_tokenize(x))

#join the list of tokens back into a single string with space-separated words.
df.reviewText = df.reviewText.apply(lambda x: ' '.join(x))

In [ ]:
def convert_to_iso(date_str):
    date_obj = datetime.strptime(date_str, '%m %d, %Y')
    return date_obj.strftime('%Y-%m-%d')

df['reviewTime'] = df['reviewTime'].apply(convert_to_iso)

In [ ]:
df['reviewTime'].unique()

In [ ]:
df['year'] =  pd.to_datetime(df['reviewTime']).dt.year
yearly_counts = df.groupby('year').size()

print(yearly_counts)
#2000-2008: 7849  
#2009      6995
#2010     13282
#2011     25834
#2012     48283
#2013    141435
#2014    102677

#2000-2011 53960 train 2014 test?

In [ ]:
df.head()

In [ ]:
df.at[1,"reviewerName"]
df.at[7,"reviewerName"] # reviewerName has repeated name and sybols like " and\ ;didn't process it because we won't use it

In [ ]:
df['helpful'].astype(str).unique()

In [ ]:
# Rename columns
df = df.rename(columns={
    "reviewerID": "uid",
    "reviewText": "text",
    "reviewTime": "time"
})


df.iloc[1213]

In [ ]:

# Write the dataframe to a JSON file
with open('amazon.json', 'w') as wfile:
    wfile.write(df.to_json(orient='records', lines=True))

video game 2018

In [2]:
vgpath='/HDD16TB/weisi/Video_Games.json.gz'

with gzip.open(vgpath,'rt') as f:
    vgdf=pd.read_json(f,lines=True)

#vgdf.head()

In [3]:
vgdf.fillna('x', inplace=True) #fills missing (NaN) values in the with the character 'x'
vgdf.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,x,x,x
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,x,x,x
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,x,x
3,1,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,x,x
4,4,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,x,x,x


In [ ]:
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
vgdf.reviewText = vgdf.reviewText.progress_apply(lambda x: preprocess.clean_and_tokenize(x))

#join the list of tokens back into a single string with space-separated words.
vgdf.reviewText = vgdf.reviewText.apply(lambda x: ' '.join(x))




In [4]:
def convert_to_iso(date_str):
    date_obj = datetime.strptime(date_str, '%m %d, %Y')
    return date_obj.strftime('%Y-%m-%d')

vgdf['reviewTime'] = vgdf['reviewTime'].apply(convert_to_iso)

vgdf['year'] =  pd.to_datetime(vgdf['reviewTime']).dt.year
yearly_counts = vgdf.groupby('year').size()

#print(yearly_counts)

In [ ]:
vgdf.head()

In [ ]:
print(vgdf.at[3246,'reviewText'])

In [ ]:
vgdf_2000 = vgdf[vgdf['year'] == 2000]
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
vgdf_2000.reviewText = vgdf_2000.reviewText.progress_apply(lambda x: preprocess.clean_and_tokenize(x))
#join the list of tokens back into a single string with space-separated words.
vgdf_2000.reviewText = vgdf_2000.reviewText.apply(lambda x: ' '.join(x))

vgdf_2000 = vgdf_2000.rename(columns={
    "overall": "label",
    "reviewText": "text",
    "reviewTime": "time"
})


vgdf_2000.to_json('amazonVG_2000.json', orient='records', lines=True)



In [12]:

vgdf_2001 = vgdf[vgdf['year'] == 2001]#.sample(n=20000)
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
vgdf_2001.reviewText = vgdf_2001.reviewText.progress_apply(lambda x: preprocess.fast_clean(x))
#get stucked at 7615,7224,7188,7201/23073... seems like error happened in re.sub(r"(\d+)+(\-)+(\d+)+(\-)+(\d+)", "date", doc), so use a new fast_clean function

#join the list of tokens back into a single string with space-separated words.
vgdf_2001.reviewText = vgdf_2001.reviewText.apply(lambda x: ' '.join(x))
vgdf_2001 = vgdf_2001.rename(columns={
    "overall": "label",
    "reviewText": "text",
    "reviewTime": "time"
})
#vgdf_2001.to_json('amazonVG_2001.json', orient='records', lines=True)
with open('amazonVG.json', 'w') as wfile:
    wfile.write(df.to_json(orient='records', lines=True))






  0%|          | 1/23073 [00:00<00:09, 2508.56it/s]


AttributeError: module 'dataproc.preprocess' has no attribute 'fast_clean'

In [ ]:
vgdf_2002 = vgdf[vgdf['year'] == 2002]#.sample(n=20000)
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
vgdf_2002.reviewText = vgdf_2002.reviewText.progress_apply(lambda x: preprocess.clean_and_tokenize(x))
#join the list of tokens back into a single string with space-separated words.
vgdf_2002.reviewText = vgdf_2002.reviewText.apply(lambda x: ' '.join(x))
vgdf_2002 = vgdf_2002.rename(columns={
    "overall": "label",
    "reviewText": "text",
    "reviewTime": "time"
})
vgdf_2002.to_json('amazonVG_2002.json', orient='records', lines=True)

In [ ]:
#print(vgdf_2001.at[7615,'reviewText'])
print(vgdf_2001.iloc[7223]['reviewText'])


In [ ]:
# randomly select 2000 data from 2018
vgdf_2018_sample = vgdf[vgdf['year'] == 2018].sample(n=2000)
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
vgdf_2018_sample.reviewText = vgdf_2018_sample.reviewText.progress_apply(lambda x: preprocess.clean_and_tokenize(x))
#join the list of tokens back into a single string with space-separated words.
vgdf_2018_sample.reviewText = vgdf_2018_sample.reviewText.apply(lambda x: ' '.join(x))

vgdf_2018_sample = vgdf_2018_sample.rename(columns={
    "overall": "label",
    "reviewText": "text",
    "reviewTime": "time"
})
vgdf_2018_sample.to_json('amazonVG_2018_samples_2k.json', orient='records', lines=True)

In [ ]:
# Write the dataframe to a JSON file
with open('amazonVG.json', 'w') as wfile:
    wfile.write(df.to_json(orient='records', lines=True))